<a href="https://colab.research.google.com/github/A-l-E-v/ML-Engineer/blob/main/request_final_only_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание
# Тема 2.2. Request

Цель: научиться парсить данные при помощи библиотеки Request.

# 1. Просто get-запрос

### Задание
С помощью библиотеки Requests отправьте get запрос на страницу https://ru.wikipedia.org/wiki/Капибара и выведите статус-код операции.

In [1]:
import requests

# Отправка GET-запроса на страницу Википедии о капибаре
response = requests.get('https://ru.wikipedia.org/wiki/Капибара')

# Вывод статус-кода операции
print(f"Статус-код: {response.status_code}")

Статус-код: 200


## 2. Парсинг википедии

### Задание
С помощью библиотеки Requests распарсите страницу https://ru.wikipedia.org/wiki/Биткойн, соберите ссылки на изображения (подсказка: атрибут src) и подписи к этим изображениям (подсказка: тег figcaption). Оформите результат в пандас-таблицу.

In [2]:
from bs4 import BeautifulSoup
import pandas as pd

# Получение HTML-страницы о Биткойне
response = requests.get('https://ru.wikipedia.org/wiki/Биткойн')
soup = BeautifulSoup(response.text, 'html.parser')

# Сбор данных об изображениях
images = []
captions = []

# Поиск всех изображений (тег figure)
for figure in soup.find_all('figure'):
    img = figure.find('img')
    if img and 'src' in img.attrs:
        images.append(img['src'])
        caption = figure.find('figcaption')
        captions.append(caption.get_text() if caption else 'Нет подписи')

# Создание DataFrame
df = pd.DataFrame({
    'Изображение': images,
    'Подпись': captions
})

# Вывод таблицы
print(df)

                                          Изображение  \
0   //upload.wikimedia.org/wikipedia/commons/thumb...   
1   //upload.wikimedia.org/wikipedia/commons/thumb...   
2   //upload.wikimedia.org/wikipedia/commons/thumb...   
3   //upload.wikimedia.org/wikipedia/ru/thumb/5/58...   
4   //upload.wikimedia.org/wikipedia/commons/thumb...   
5   //upload.wikimedia.org/wikipedia/commons/thumb...   
6   //upload.wikimedia.org/wikipedia/commons/thumb...   
7   //upload.wikimedia.org/wikipedia/commons/thumb...   
8   //upload.wikimedia.org/wikipedia/commons/thumb...   
9   //upload.wikimedia.org/wikipedia/commons/thumb...   
10  //upload.wikimedia.org/wikipedia/commons/thumb...   
11  //upload.wikimedia.org/wikipedia/commons/thumb...   
12  //upload.wikimedia.org/wikipedia/commons/thumb...   
13  //upload.wikimedia.org/wikipedia/commons/thumb...   
14  //upload.wikimedia.org/wikipedia/commons/thumb...   
15  //upload.wikimedia.org/wikipedia/commons/thumb...   
16  //upload.wikimedia.org/wiki

## 3. Краулинг форума

### Задание
Распарсите все страницы из раздела "Ветеринария КРС" https://fermer.ru/forum/zhivotnovodstvo/krupnyi-rogatyi-skot/veterinariya-krs. С каждой страницы соберите таблицу и сохраните из нее следующую информацию: имя темы, ссылка на тему, количество оветов, количество просмотров, имя автора темы.

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

base_url = 'https://fermer.ru/forum/zhivotnovodstvo/krupnyi-rogatyi-skot/veterinariya-krs'
all_data = []

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def parse_page(page_url):
    try:
        response = requests.get(page_url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        topics_table = soup.find('table', class_='forum-table-topics')
        
        if not topics_table:
            print(f"Не удалось найти таблицу тем на странице {page_url}")
            return
        
        for row in topics_table.find_all('tr')[1:]:
            try:
                cols = row.find_all('td')
                if len(cols) < 5:
                    continue
                
                # Находим ссылку с названием темы (может быть вложенной)
                topic_link_tag = cols[1].find('a', href=True)
                topic_name = ""
                
                # Ищем название темы в разных вариантах
                title_span = cols[1].find('span', class_='forum-topic-title')
                if title_span:
                    topic_name = title_span.text.strip()
                elif topic_link_tag:
                    topic_name = topic_link_tag.text.strip()
                
                # Если название не найдено, пропускаем тему
                if not topic_name:
                    continue
                
                # Получаем ссылку
                topic_link = topic_link_tag['href'] if topic_link_tag else "#"
                
                # Автор и дата
                author_info = cols[1].find('a', class_='username')
                author = author_info.text.strip() if author_info else "Неизвестно"
                
                # Ответы и просмотры
                replies = cols[2].text.strip()
                views = cols[3].text.strip().replace(',', '')  # Удаляем запятые в числах
                
                # Последнее обновление
                last_update = cols[4].get_text(separator=' ', strip=True)
                
                all_data.append({
                    'Тема': topic_name,
                    'Ссылка': f"https://fermer.ru{topic_link}" if not topic_link.startswith('http') else topic_link,
                    'Ответы': replies,
                    'Просмотры': views,
                    'Автор': author,
                    'Последнее обновление': last_update
                })
                
            except Exception as e:
                print(f"Ошибка при парсинге строки: {e}")
                continue
                
    except Exception as e:
        print(f"Ошибка при запросе страницы {page_url}: {e}")

# Парсим первые 5 страниц
for page_num in range(0, 5):
    page_url = f"{base_url}?page={page_num}"
    print(f"Парсинг страницы: {page_url}")
    parse_page(page_url)
    time.sleep(2)

# Создаем DataFrame
if all_data:
    forum_df = pd.DataFrame(all_data)
    print(forum_df)
    
    # Сохраняем в CSV
    forum_df.to_csv('fermer_forum_topics.csv', index=False, encoding='utf-8-sig')
    print(f"Сохранено {len(forum_df)} тем в fermer_forum_topics.csv")
else:
    print("Не удалось получить данные. Проверьте структуру сайта.")

Парсинг страницы: https://fermer.ru/forum/zhivotnovodstvo/krupnyi-rogatyi-skot/veterinariya-krs?page=0
Парсинг страницы: https://fermer.ru/forum/zhivotnovodstvo/krupnyi-rogatyi-skot/veterinariya-krs?page=1
Парсинг страницы: https://fermer.ru/forum/zhivotnovodstvo/krupnyi-rogatyi-skot/veterinariya-krs?page=2
Парсинг страницы: https://fermer.ru/forum/zhivotnovodstvo/krupnyi-rogatyi-skot/veterinariya-krs?page=3
Парсинг страницы: https://fermer.ru/forum/zhivotnovodstvo/krupnyi-rogatyi-skot/veterinariya-krs?page=4
                                                  Тема  \
0    Массовый падёж коров. При вскрытии - страшная ...   
1    Годовалый бык сильно худеет. Ноги трясутся, ес...   
2      На задней ноге быка шишка. Фото  Опасно ли это?   
3    В хозяйстве стали рождаться от нетелей телята ...   
4    Обнаружил у коровы в районе правой передней ло...   
..                                                 ...   
120                         Корова выплёвывает жвачку.   
121                  

## 4. Тестовое API

### Задание
Получите данные последних 6 пользователей с сайта https://fakestoreapi.com с помощью API.  
Подсказка: используйте параметр sort.

In [4]:
import requests
import pandas as pd

# Получение данных последних 6 пользователей
response = requests.get('https://fakestoreapi.com/users?sort=desc&limit=6')
users = response.json()

# Создание DataFrame
users_df = pd.DataFrame(users)

# Вывод данных
print(users_df[['id', 'name', 'username', 'email']])

   id                                             name  username  \
0  10     {'firstname': 'jimmie', 'lastname': 'klein'}  jimmie_k   
1   9        {'firstname': 'kate', 'lastname': 'hale'}    kate_h   
2   8  {'firstname': 'william', 'lastname': 'hopkins'}   hopkins   
3   7    {'firstname': 'miriam', 'lastname': 'snyder'}    snyder   
4   6    {'firstname': 'david', 'lastname': 'russell'}   david_r   
5   5     {'firstname': 'derek', 'lastname': 'powell'}     derek   

               email  
0   jimmie@gmail.com  
1     kate@gmail.com  
2  william@gmail.com  
3   miriam@gmail.com  
4  david_r@gmail.com  
5    derek@gmail.com  


**Результат:** решение для каждого задания предоставляется в виде программного кода на языке Python

**Критерии оценивания:**

К1 - отправлен get запрос на сайт из Задания 1, выведен статус-код операции (4 балла)

К2 - распарсена страница сайта из Задания 2, собраны  ссылки на изображения и подписи к этим изображениям. Собранные данные представлены в виде пандас-таблицы (6 баллов)

К3 - распарсены все страницы из раздела "Ветеринария КРС" из Задания 3, с каждой страницы взята таблица и из нее сохранена соответствующая условию задания ифнормация (6 баллов)

К4 -  получены данные последних 6 пользователей с сайта из Задания 4 с помощью API (4 балла)


Максимальное количество баллов —  20 баллов.

Минимальное количество баллов, чтобы преподаватель смог зачесть вашу работу — 10 баллов.